In [1]:
import pandas as pd

In [2]:
!pip install wandb -qqq
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.9 MB/s eta 0:00:00


In [4]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 9.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
!wget https://github.com/Aszanat/2024-emotion-recognition-v2/raw/main/train/in.tsv
!wget https://github.com/Aszanat/2024-emotion-recognition-v2/raw/main/train/expected.tsv

input = pd.read_csv('in.tsv', sep='\t').dropna()
output = pd.read_csv('expected.tsv', sep='\t').dropna()

--2024-07-05 13:51:08--  https://github.com/Aszanat/2024-emotion-recognition-v2/raw/main/train/in.tsv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Aszanat/2024-emotion-recognition-v2/main/train/in.tsv [following]
--2024-07-05 13:51:08--  https://raw.githubusercontent.com/Aszanat/2024-emotion-recognition-v2/main/train/in.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 839457 (820K) [text/plain]
Saving to: ‘in.tsv’

in.tsv              100%[===================>] 819.78K  --.-KB/s    in 0.006s  

2024-07-05 13:51:09 (133 MB/s) - ‘in.tsv’ saved [839457/839457]

--2024-07-05 13:51:09--  ht

In [6]:
def divide_opinions(table):
    opinions = []
    sentences = []
    for i, j in table.iterrows():
      if (j.get('text') == '###########################'):
        opinions.append(sentences)
        sentences = []
      else:
        sentences.append(j.get('text'))
    return opinions

In [7]:
opinions = divide_opinions(input)

In [8]:
def opinions_to_strings(opinions):
  # ops will be a list of string, each string being a whole opinion with the sentence to evaluate at the end (or just the opinion to evaluate)
  ops = []
  for o in opinions:
    review = ''.join(o)
    for s in o:
      ops.append(review + s)
    ops.append(review)
  return ops

In [9]:
sentences = opinions_to_strings(opinions)

In [10]:
def labels_to_array(label_table):
  lab_vecs = []
  for i, j in label_table.iterrows():
    lab_vec = []
    for value in j.array:
      if (value):
        lab_vec.append(1)
      else:
        lab_vec.append(0)
    lab_vecs.append(lab_vec)
  return lab_vecs

In [13]:
import torch
from torch.nn import BCEWithLogitsLoss
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, EvalPrediction
from sklearn.model_selection import train_test_split

In [ ]:
labels = torch.tensor(labels_to_array(output))

num_labels = 11  # Adjust based on your dataset

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("allegro/herbert-base-cased", num_labels=num_labels)

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Define accuracy calculation function
def multilabel_accuracy(predictions, labels, threshold=0.5):
    """
    Calculate accuracy for multilabel classification.

    Args:
    predictions: torch.Tensor, model predictions (logits).
    labels: torch.Tensor, true labels.
    threshold: float, threshold to convert probabilities to binary predictions.

    Returns:
    float, accuracy score.
    """
    # Apply sigmoid to get probabilities
    probs = torch.sigmoid(predictions)

    # Threshold probabilities to get binary predictions
    binary_preds = (probs > threshold).float()

    # Compare binary predictions with true labels
    correct = (binary_preds == labels).float()

    # Calculate accuracy for each label
    accuracy_per_label = correct.mean(dim=0)

    # Calculate average accuracy across all labels
    average_accuracy = accuracy_per_label.mean().item()

    return average_accuracy

def compute_metrics(p: EvalPrediction):
    preds = torch.tensor(p.predictions)
    labels = torch.tensor(p.label_ids)
    accuracy = multilabel_accuracy(preds, labels)
    return {"accuracy": accuracy}

# Prepare the dataset
class MultilabelDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Split dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(sentences, labels, test_size=0.2)
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
train_dataset = MultilabelDataset(train_encodings, train_labels)
val_dataset = MultilabelDataset(val_encodings, val_labels)

# Custom Trainer to handle multilabel loss
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())
        return (loss, outputs) if return_outputs else loss

# Prepare training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    num_train_epochs=3,
    learning_rate=0.00002,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.001,
    disable_tqdm=False,
    overwrite_output_dir=True,
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
    save_strategy='epoch',
    logging_strategy='epoch',
    log_level='error',
    warmup_ratio=0.05,
    report_to='wandb'
)

# Create the custom trainer
trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train and evaluate
wandb.init(project='HerBERTEmotionRecognition')
trainer.train()
results = trainer.evaluate()
wandb.log(results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
eval/loss,0.27556
eval/runtime,43.4799


<ipython-input-14-e0b858613ace>:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss
